In [1]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./1.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
input_['Category'].tolist()

总数量：26


['Apparel & Gifts',
 'Belt Drive',
 'Body & Lamp Assembly',
 'Brake & Wheel Hub',
 'Cooling System',
 'Drivetrain',
 'Electrical',
 'Electrical-Bulb & Socket',
 'Electrical-Connector',
 'Electrical-Switch & Relay',
 'Engine',
 'Exhaust & Emission',
 'Fuel & Air',
 'Garage Equipment',
 'Hardware',
 'Heat & Air Conditioning',
 'Hoses/Lines & Clamps',
 'Ignition',
 'Interior',
 'Literature',
 'Steering',
 'Suspension',
 'Transmission-Automatic',
 'Transmission-Manual',
 'Wheel',
 'Wiper & Washer']

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
from urllib.parse import quote
import requests
import json

In [5]:
i = 0

jsn = '{"jsn":{"tab":"extras","groupname":"' + input_.loc[i, 'Category'] + '","nodetype":"groupname"}}'

payload = {'func': 'navnode_fetch',
           'payload': jsn,
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload).text

resp = json.loads(resp)

resp

{'html_fill_sections': {'navchildren[]': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;extras&quot;,&quot;idepth&quot;:1,&quot;groupname&quot;:&quot;Apparel &amp; Gifts&quot;,&quot;subgroupname&quot;:&quot;RockAuto Apparel&quot;,&quot;nodetype&quot;:&quot;subgroupname&quot;}" /><input type="hidden" autocomplete="off" name="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" id="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nvlineicon"></td><td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]"><a href="/en/tools/apparel+&amp;+gifts,rockauto+apparel" onclick="cataloglite.LinkIntercept_ToggleNavNode(&quot;__GIP__1__&quot;); return false;" tabindex="-1"><img src="

In [6]:
from bs4 import BeautifulSoup

In [7]:
html = resp['html_fill_sections']['navchildren[]']

soup = BeautifulSoup(html, 'lxml')

with open('./subcategory.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"extras","idepth":1,"groupname":"Apparel &amp; Gifts","subgroupname":"RockAuto Apparel","nodetype":"subgroupname"}'/>
   <input autocomplete="off" id="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" name="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nvlineicon">
       </td>
       <td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]">
        <a href="/en/tools/apparel+&amp;+gifts,rockauto+apparel" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
        </a>
       </td>
      

In [8]:
from lxml import etree

In [9]:
html = etree.HTML(str(soup))

list_subcategory = [subcategory.strip() for subcategory in html.xpath('//td[@class="nlabel"]/a/text()')]

list_subcategory

['RockAuto Apparel', 'RockAuto Gift']

In [10]:
list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]

list_url

['https://www.rockauto.com/en/tools/apparel+&+gifts,rockauto+apparel',
 'https://www.rockauto.com/en/tools/apparel+&+gifts,rockauto+gift']

In [11]:
crawler_status = 'ok'

print(crawler_status)

correct


In [12]:
if crawler_status == 'ok':
    df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                            'No.': int(input_.loc[i, 'No.']),
                            'Category': input_.loc[i, 'Category'],
                            'Subcategory': list_subcategory,
                            'Url': list_url})
    
    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
else:
    df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                             'No.': int(input_.loc[i, 'No.']),
                             'Category': input_.loc[i, 'Category'],
                             'Url': input_.loc[i, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
print(len(output_correct), len(output_error))

2 0
